In [4]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TestApp") \
    .getOrCreate()

print(spark.version)


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark


3.5.1


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
import plotly.express as px

In [6]:
df = spark.read.format('csv').option('header', 'true').load('challenge.csv')
df.show()

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

In [8]:
Country_filter = ["Mexico"]

df_Country  = df.filter(df.Country.isin(Country_filter))


In [9]:
df_Country.show()

+--------------+-------+-------------+----------+
|    ip_address|Country|  Domain Name|Bytes_used|
+--------------+-------+-------------+----------+
|     5.72.7.65| Mexico|     youtu.be|       877|
|132.36.204.150| Mexico| symantec.com|       539|
| 83.174.191.91| Mexico|    unblog.fr|       226|
|  3.162.25.151| Mexico|   flickr.com|       860|
|68.190.125.173| Mexico| virginia.edu|       158|
| 72.73.135.209| Mexico|  foxnews.com|       113|
| 127.149.93.62| Mexico|   sbwire.com|       588|
| 84.217.103.71| Mexico| newsvine.com|       397|
|  87.165.14.11| Mexico| buzzfeed.com|       661|
|237.35.182.200| Mexico| redcross.org|       824|
| 255.45.217.54| Mexico|newyorker.com|       597|
| 196.17.181.29| Mexico| china.com.cn|       433|
| 166.53.113.35| Mexico|      home.pl|        20|
+--------------+-------+-------------+----------+



In [10]:
df_with_filter = df.withColumn('is_Mexico', when(df['Country'] == 'Mexico', lit('yes')).otherwise(lit('no')))

df_with_filter.show()

+---------------+--------------+-----------------+----------+---------+
|     ip_address|       Country|      Domain Name|Bytes_used|is_Mexico|
+---------------+--------------+-----------------+----------+---------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|       no|
| 119.239.207.13|         China|         youtu.be|        51|       no|
|  68.69.217.210|         China|        adobe.com|        10|       no|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|       no|
|   211.13.10.68|     Indonesia|          hud.gov|        29|       no|
|   239.80.21.97|      Suriname|       smh.com.au|       218|       no|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|       no|
| 127.242.24.138|         China| surveymonkey.com|       123|       no|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|       no|
|   237.54.11.63|         China|       amazon.com|        83|       no|
| 252.141.157.25|         Japan|      cornell.edu|       374|   

In [36]:
df = df.withColumn('is_Mexico', when(df['Country'] == 'Mexico', lit('yes')).otherwise(lit('no')))

# Group by 'is_Mexico' and sum 'Bytes_used'
grouped_df = df_with_filter.groupBy('is_Mexico').agg(sum('Bytes_used').alias('total_bytes_used'))


grouped_df.show()

+---------+----------------+
|is_Mexico|total_bytes_used|
+---------+----------------+
|       no|        508076.0|
|      yes|          6293.0|
+---------+----------------+



In [38]:
# Group by 'Country' and count distinct 'ip_address'
country_ip_count_df = df.groupBy('Country').agg(countDistinct('ip_address').alias('distinct_ip_count'))

country_ip_count_df.show()

CodeCache: size=131072Kb used=32073Kb max_used=32091Kb free=98998Kb
 bounds [0x00000001099e8000, 0x000000010b978000, 0x00000001119e8000]
 total_blobs=11954 nmethods=11013 adapters=852
 compilation: disabled (not enough contiguous free space left)
+-----------+-----------------+
|    Country|distinct_ip_count|
+-----------+-----------------+
|       Chad|                1|
|     Russia|               56|
|   Paraguay|                1|
|      Yemen|                1|
|     Sweden|               28|
|Philippines|               65|
|   Malaysia|                5|
|     Turkey|                1|
|     Malawi|                2|
|    Germany|                5|
|    Comoros|                1|
|Afghanistan|                5|
|     Rwanda|                1|
|      Sudan|                1|
|     France|               21|
|     Greece|                8|
|  Sri Lanka|                3|
|   Dominica|                1|
|  Argentina|               14|
|    Belgium|                1|
+-----------+----

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


In [42]:
country_ip_count_pd = country_ip_count_df.toPandas()

# Get the top 10 countries by IP
top_10_countries = country_ip_count_pd.sort_values(by='distinct_ip_count', ascending=False).head(10)

# Plot the results using Plotly
fig = px.bar(top_10_countries, x='Country', y='distinct_ip_count', color='Country',
             title='Top 10 countries that accessed the Web Service',
             labels={'distinct_ip_count': 'Number of Distinct IP Addresses'})

# Show the plot
fig.show()

In [41]:
#Identify which country accesed most frequently
most_frequent_country = df.groupBy('Country').count().orderBy(desc('count')).first()['Country']

#Filter data for Japanese 
hacker_df = df.filter((df['Country'] == most_frequent_country) &
                      (df['Domain Name'].contains('.jp')) &
                      (df['Bytes_used'].between(800, 900)))

# Select IP of hacker
hacker_ip = hacker_df.select('ip_address').distinct().collect()

print("The hacker's IP address is:", hacker_ip)

The hacker's IP address is: [Row(ip_address='154.34.244.199')]


In [43]:
hacker_df.show()

+--------------+-------+------------+----------+---------+
|    ip_address|Country| Domain Name|Bytes_used|is_Mexico|
+--------------+-------+------------+----------+---------+
|154.34.244.199|  China|amazon.co.jp|       897|       no|
+--------------+-------+------------+----------+---------+



24/05/27 19:25:18 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 941647 ms exceeds timeout 120000 ms
24/05/27 19:25:18 WARN SparkContext: Killing executors is not supported by current scheduler.
24/05/27 19:40:41 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$